The notebook seek to the anaysis of bbl season 23-24

In [24]:
import numpy as np

In [2]:
# filtering the required json files

import os
import json
from datetime import datetime
import shutil

folder_path = "bbl_json"
output_folder = "bbl"
os.makedirs(output_folder, exist_ok=True)

start_date = datetime(2023, 12, 1)
end_date = datetime(2024, 1, 31)

for filename in os.listdir(folder_path):
    if filename.endswith(".json"):
        file_path = os.path.join(folder_path, filename)
        
        with open(file_path, "r") as file:
            data = json.load(file)
            
            created_date_str = data.get("meta", {}).get("created", "")
            
            try:
                created_date = datetime.strptime(created_date_str, "%Y-%m-%d")
                if start_date <= created_date <= end_date:
                    shutil.copy(file_path, output_folder)
                    print(f"Copied {filename}")
            except ValueError:
                print(f"Skipping {filename} due to invalid date format.")


Copied 1386094.json
Copied 1386095.json
Copied 1386097.json
Copied 1386098.json
Copied 1386099.json
Copied 1386100.json
Copied 1386101.json
Copied 1386102.json
Copied 1386103.json
Copied 1386104.json
Copied 1386105.json
Copied 1386106.json
Copied 1386107.json
Copied 1386108.json
Copied 1386109.json
Copied 1386110.json
Copied 1386111.json
Copied 1386112.json
Copied 1386113.json
Copied 1386114.json
Copied 1386116.json
Copied 1386117.json
Copied 1386118.json
Copied 1386119.json
Copied 1386120.json
Copied 1386121.json
Copied 1386122.json
Copied 1386123.json
Copied 1386124.json
Copied 1386125.json
Copied 1386126.json
Copied 1386127.json
Copied 1386128.json
Copied 1386129.json
Copied 1386130.json
Copied 1386131.json
Copied 1386132.json
Copied 1386134.json
Copied 1386135.json
Copied 1386136.json
Copied 1386137.json


### Statistical analysis and feautures

In [3]:
import os
import json
import pandas as pd
from tqdm import tqdm

bbl_folder = 'bbl/'

matches_data = []

for file in tqdm(os.listdir(bbl_folder), desc="Loading and Parsing JSON Files"):
    if file.endswith('.json'):
        with open(os.path.join(bbl_folder, file), 'r') as f:
            match = json.load(f)
            
            match_info = {
                'match_id': match.get('meta', {}).get('id'),
                'venue': match.get('info', {}).get('venue'),
                'city': match.get('info', {}).get('city'),
                'season': match.get('info', {}).get('season'),
                'teams': match.get('info', {}).get('teams', []),
                'winner': match.get('info', {}).get('outcome', {}).get('winner'),
                'outcome_type': list(match.get('info', {}).get('outcome', {}).get('by', {}).keys())[0]
                               if match.get('info', {}).get('outcome', {}).get('by') else None,
                'outcome_margin': list(match.get('info', {}).get('outcome', {}).get('by', {}).values())[0]
                               if match.get('info', {}).get('outcome', {}).get('by') else None,
            }
            matches_data.append(match_info)

matches_df = pd.DataFrame(matches_data)
matches_df.head()

Loading and Parsing JSON Files: 100%|██████████████████████████████████████████████████| 41/41 [00:00<00:00, 62.12it/s]


,match_id,venue,city,season,teams,winner,outcome_type,outcome_margin
0,None,"Brisbane Cricket Ground, Woolloongabba, Brisbane",Brisbane,2023/24,"[Brisbane Heat, Melbourne Stars]",Brisbane Heat,runs,103.0
1,None,Sydney Cricket Ground,Sydney,2023/24,"[Sydney Sixers, Melbourne Renegades]",Sydney Sixers,runs,8.0
2,None,"GMHBA Stadium, South Geelong, Victoria",Geelong,2023/24,"[Perth Scorchers, Melbourne Renegades]",None,None,NaN
3,None,"University of Tasmania Stadium, Launceston",Launceston,2023/24,"[Hobart Hurricanes, Sydney Sixers]",Sydney Sixers,wickets,6.0
4,None,"Manuka Oval, Canberra",Canberra,2023/24,"[Brisbane Heat, Sydney Thunder]",Brisbane Heat,runs,20.0


In [4]:
# Add 'is_draw' column to indicate if the match was a draw
matches_df['is_draw'] = matches_df['winner'].apply(lambda x: 1 if pd.isna(x) else 0)

# Display the updated DataFrame with the new 'is_draw' column
print(matches_df[['match_id', 'winner', 'is_draw']].head())


  match_id         winner  is_draw
0     None  Brisbane Heat        0
1     None  Sydney Sixers        0
2     None           None        1
3     None  Sydney Sixers        0
4     None  Brisbane Heat        0


In [8]:
# Step 1: Calculate Wins
# Count occurrences of each team as a winner
wins_df = matches_df['winner'].value_counts().reset_index()
wins_df.columns = ['team', 'wins']

# Step 2: Calculate Draws
# Filter matches with is_draw == 1, explode the teams, and count each team in a draw
draws_df = matches_df[matches_df['is_draw'] == 1].explode('teams')
draws_df = draws_df['teams'].value_counts().reset_index()
draws_df.columns = ['team', 'draws']

# Step 3: Total Matches Played per Team
# Explode teams to get individual team entries for each match
all_teams = matches_df[['match_id', 'teams']].explode('teams')
all_teams = all_teams['teams'].value_counts().reset_index()
all_teams.columns = ['team', 'total_games']

# Step 4: Merge Wins, Draws, and Total Games to Calculate Losses
# Merge wins, draws, and total games into a single DataFrame
team_stats_df = all_teams.merge(wins_df, on='team', how='left').merge(draws_df, on='team', how='left')

# Fill NaN values with 0 for teams that may not have wins or draws
team_stats_df[['wins', 'draws']] = team_stats_df[['wins', 'draws']].fillna(0).astype(int)

# Calculate losses as total games - wins - draws
team_stats_df['losses'] = team_stats_df['total_games'] - team_stats_df['wins'] - team_stats_df['draws']

# Display the result
print("Team Win/Loss/Draw Statistics:")
print(team_stats_df[['team', 'wins', 'losses', 'draws']])


Team Win/Loss/Draw Statistics:
                  team  wins  losses  draws
0        Brisbane Heat     9       2      0
1        Sydney Sixers     7       3      1
2      Perth Scorchers     6       4      1
3    Adelaide Strikers     6       5      0
4      Melbourne Stars     4       6      0
5    Hobart Hurricanes     4       6      0
6  Melbourne Renegades     2       6      1
7       Sydney Thunder     1       7      1


In [12]:
# Calculate the distribution of games by city
city_distribution = matches_df['city'].value_counts().reset_index()
city_distribution.columns = ['city', 'games_played']

print("\nGame Distribution by City:")
print(city_distribution)



Game Distribution by City:
             city  games_played
0       Melbourne             8
1          Sydney             7
2           Perth             6
3        Brisbane             4
4        Adelaide             4
5          Hobart             4
6        Canberra             2
7         Carrara             2
8         Geelong             1
9      Launceston             1
10         Albury             1
11  Coffs Harbour             1


In [13]:
# Calculate wins by stadium and team
stadium_wins = matches_df.groupby(['venue', 'winner']).size().unstack(fill_value=0)

# Calculate losses by stadium
# First, repeat teams for each match, so we have both teams per game
matches_with_teams = matches_df[['match_id', 'venue', 'teams', 'winner']].explode('teams')
matches_with_teams = matches_with_teams[matches_with_teams['teams'] != matches_with_teams['winner']]
losses_by_stadium = matches_with_teams.groupby(['venue', 'teams']).size().unstack(fill_value=0)

# Rename columns
stadium_wins.index.name = 'stadium'
stadium_wins.columns.name = 'team'
losses_by_stadium.index.name = 'stadium'
losses_by_stadium.columns.name = 'team'

print("\nWins by Stadium and Team:")
print(stadium_wins)

print("\nLosses by Stadium and Team:")
print(losses_by_stadium)



Wins by Stadium and Team:
team                                              Adelaide Strikers  \
stadium                                                               
Adelaide Oval                                                     3   
Bellerive Oval, Hobart                                            1   
Brisbane Cricket Ground, Woolloongabba, Brisbane                  0   
Carrara Oval                                                      0   
Docklands Stadium, Melbourne                                      0   
International Sports Stadium, Coffs Harbour                       0   
Lavington Sports Oval, Albury                                     0   
Manuka Oval, Canberra                                             1   
Melbourne Cricket Ground                                          0   
Perth Stadium                                                     1   
Sydney Cricket Ground                                             0   
Sydney Showground Stadium                         

In [18]:

matches_dir = 'bbl/'  # Update this to your directory containing JSON files

# Initialize an empty list to store each delivery's data
ball_by_ball_data = []

# Iterate through each match JSON file
for file in tqdm(os.listdir(matches_dir), desc="Processing Matches"):
    if file.endswith('.json'):
        with open(os.path.join(matches_dir, file), 'r') as f:
            match = json.load(f)
        
        # Extract teams for this match
        teams = match['info']['teams']
        team1, team2 = teams[0], teams[1]
        
        # Process each inning
        for inning_number, inning in enumerate(match['innings']):
            batting_team = inning['team']
            bowling_team = team2 if batting_team == team1 else team1  # Opposite team is the bowling team
            
            # Process each over
            for over_info in inning['overs']:
                over = over_info['over']
                
                # Process each delivery
                for ball_number, delivery_info in enumerate(over_info['deliveries'], start=1):
                    # Basic delivery info
                    batsman = delivery_info.get('batter')
                    non_striker = delivery_info.get('non_striker')
                    bowler = delivery_info.get('bowler')
                    
                    # Runs information
                    runs = delivery_info['runs']
                    batsman_runs = runs.get('batter', 0)
                    extra_runs = runs.get('extras', 0)
                    total_runs = runs.get('total', 0)
                    
                    # Non-boundary indicator
                    non_boundary = 1 if batsman_runs < 4 else 0  # Assume non-boundary if runs < 4
                    
                    # Extras information
                    extras_type = list(delivery_info.get('extras', {}).keys())[0] if 'extras' in delivery_info else None
                    
                    # Wicket information
                    is_wicket = 1 if 'wickets' in delivery_info else 0
                    dismissal_kind = None
                    player_dismissed = None
                    fielder = None
                    
                    if is_wicket:
                        for wicket in delivery_info['wickets']:
                            dismissal_kind = wicket.get('kind')
                            player_dismissed = wicket.get('player_out')
                            fielder = ', '.join([fielder['name'] for fielder in wicket.get('fielders', [])]) if wicket.get('fielders') else None
                    
                    # Append data to the list
                    ball_by_ball_data.append({
                        'id': file.replace('.json', ''),  # Match identifier from filename
                        'inning': inning_number + 1,     # Inning number (1 or 2)
                        'over': over,
                        'ball': ball_number,
                        'batsman': batsman,
                        'non_striker': non_striker,
                        'bowler': bowler,
                        'batsman_runs': batsman_runs,
                        'extra_runs': extra_runs,
                        'total_runs': total_runs,
                        'non_boundary': non_boundary,
                        'is_wicket': is_wicket,
                        'dismissal_kind': dismissal_kind,
                        'player_dismissed': player_dismissed,
                        'fielder': fielder,
                        'extras_type': extras_type,
                        'batting_team': batting_team,
                        'bowling_team': bowling_team
                    })

# Convert the list of deliveries to a DataFrame
ball_by_ball_df = pd.DataFrame(ball_by_ball_data)

# Save to CSV
ball_by_ball_df.to_csv('ball_by_ball_data.csv', index=False)

print("Ball-by-ball data has been saved to 'ball_by_ball_data.csv'.")


Processing Matches: 100%|█████████████████████████████████████████████████████████████| 42/42 [00:00<00:00, 325.78it/s]


Ball-by-ball data has been saved to 'ball_by_ball_data.csv'.


In [19]:
ball_by_ball_df.head()

,id,inning,over,ball,batsman,non_striker,bowler,batsman_runs,extra_runs,total_runs,non_boundary,is_wicket,dismissal_kind,player_dismissed,fielder,extras_type,batting_team,bowling_team
0,1386094,1,0,1,UT Khawaja,C Munro,OP Stone,2,0,2,1,0,None,None,None,None,Brisbane Heat,Melbourne Stars
1,1386094,1,0,2,UT Khawaja,C Munro,OP Stone,0,0,0,1,0,None,None,None,None,Brisbane Heat,Melbourne Stars
2,1386094,1,0,3,UT Khawaja,C Munro,OP Stone,0,0,0,1,0,None,None,None,None,Brisbane Heat,Melbourne Stars
3,1386094,1,0,4,UT Khawaja,C Munro,OP Stone,1,0,1,1,0,None,None,None,None,Brisbane Heat,Melbourne Stars
4,1386094,1,0,5,C Munro,UT Khawaja,OP Stone,0,0,0,1,0,None,None,None,None,Brisbane Heat,Melbourne Stars


In [20]:
ball_by_ball_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9277 entries, 0 to 9276
Data columns (total 18 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   id                9277 non-null   object
 1   inning            9277 non-null   int64 
 2   over              9277 non-null   int64 
 3   ball              9277 non-null   int64 
 4   batsman           9277 non-null   object
 5   non_striker       9277 non-null   object
 6   bowler            9277 non-null   object
 7   batsman_runs      9277 non-null   int64 
 8   extra_runs        9277 non-null   int64 
 9   total_runs        9277 non-null   int64 
 10  non_boundary      9277 non-null   int64 
 11  is_wicket         9277 non-null   int64 
 12  dismissal_kind    511 non-null    object
 13  player_dismissed  511 non-null    object
 14  fielder           389 non-null    object
 15  extras_type       464 non-null    object
 16  batting_team      9277 non-null   object
 17  bowling_team  

In [22]:
most_runs = ball_by_ball_df.groupby("batsman").sum()
most_runs[["batsman_runs"]].sort_values("batsman_runs", ascending = False)[:10]

,batsman_runs
batsman,
MW Short,541
J Brown,366
AM Hardie,334
CA Lynn,304
LJ Evans,292
BJ Webster,262
BR McDermott,261
AD Hales,258
JR Philippe,257


In [26]:
most_runs["batting_average"] = round(most_runs["batsman_runs"] / most_runs["is_wicket"], 2)
bat_average = most_runs.sort_values("batting_average", ascending = False)
best_batsmen = bat_average.loc[(bat_average["batting_average"] < np.inf) & (bat_average["batsman_runs"] > 100)][:10]
best_batsmen[["batsman_runs", "batting_average"]]

,batsman_runs,batting_average
batsman,,
CA Lynn,304,60.80
MW Short,541,60.11
LJ Evans,292,58.40
SE Marsh,181,45.25
BJ Webster,262,43.67
BR McDermott,261,43.50
HWR Cartwright,209,41.80
J Brown,366,40.67
J Weatherald,202,40.40


In [27]:
runs_scored_off_bowler = ball_by_ball_df.groupby("bowler").sum()
runs_scored_off_bowler["batsman_runs"]

bowler
A Zampa         248
AC Agar         259
AJ Hosein        18
AJ Turner         1
AJ Tye          271
               ... 
Usama Mir       111
W Sutherland    186
WA Agar         130
XC Bartlett     287
Zaman Khan      127
Name: batsman_runs, Length: 82, dtype: int64

In [30]:
extra_runs = ball_by_ball_df.loc[(ball_by_ball_df["extras_type"] == "wides") |
                                      (ball_by_ball_df["extras_type"] == "noballs")]
extra_runs_bowler = extra_runs.groupby("bowler").sum()
extra_runs_bowler["extra_runs"]

bowler
A Zampa          2
AC Agar          6
AJ Tye           5
AM Hardie        4
B Stanlake       3
                ..
Usama Mir        2
W Sutherland    11
WA Agar          5
XC Bartlett      7
Zaman Khan       4
Name: extra_runs, Length: 65, dtype: int64

In [31]:
# Runs scored off bowler by batsmen plus extras conceded
total_bowler_runs = runs_scored_off_bowler["batsman_runs"] + extra_runs_bowler["extra_runs"]
total_bowler_runs

bowler
A Zampa         250.0
AC Agar         265.0
AJ Hosein         NaN
AJ Turner         NaN
AJ Tye          276.0
                ...  
Usama Mir       113.0
W Sutherland    197.0
WA Agar         135.0
XC Bartlett     294.0
Zaman Khan      131.0
Length: 82, dtype: float64

In [32]:
is_dismissal = ball_by_ball_df.loc[ball_by_ball_df["is_wicket"] == 1]
is_dismissal["dismissal_kind"].unique()

array(['caught', 'lbw', 'bowled', 'run out', 'caught and bowled',
       'stumped', 'retired hurt'], dtype=object)

In [33]:
dismissal_for_bowler = is_dismissal["dismissal_kind"].str.contains("caught|bowled|lbw|stumped|hit wicket|caught and bowled'")
is_dismissal[dismissal_for_bowler]["dismissal_kind"].unique()

array(['caught', 'lbw', 'bowled', 'caught and bowled', 'stumped'],
      dtype=object)

In [34]:
wickets = is_dismissal[dismissal_for_bowler].groupby("bowler").sum()
wickets["is_wicket"]

bowler
A Zampa          6
AC Agar          6
AJ Hosein        2
AJ Tye          12
AM Hardie        6
                ..
Usama Mir        5
W Sutherland     7
WA Agar          2
XC Bartlett     20
Zaman Khan       8
Name: is_wicket, Length: 68, dtype: int64

In [37]:

legit_deliveries = ball_by_ball_df[~ball_by_ball_df['extras_type'].isin(['wides', 'no_ball'])]

bowler_stats = legit_deliveries.groupby('bowler').agg(
    balls_bowled=('ball', 'size'),          
    runs_conceded=('total_runs', 'sum')     
).reset_index()

bowler_stats['overs_bowled'] = bowler_stats['balls_bowled'] // 6 + (bowler_stats['balls_bowled'] % 6) / 10

bowler_stats['economy_rate'] = bowler_stats['runs_conceded'] / bowler_stats['overs_bowled']


# print(bowler_stats[['bowler', 'balls_bowled', 'overs_bowled', 'runs_conceded', 'economy_rate']])
bowler_stats


,bowler,balls_bowled,runs_conceded,overs_bowled,economy_rate
0,A Zampa,180,249,30.0,8.300000
1,AC Agar,192,260,32.0,8.125000
2,AJ Hosein,24,18,4.0,4.500000
3,AJ Turner,1,1,0.1,10.000000
4,AJ Tye,209,279,34.5,8.086957
...,...,...,...,...,...
77,Usama Mir,102,112,17.0,6.588235
78,W Sutherland,143,192,23.5,8.170213
79,WA Agar,79,131,13.1,10.000000
80,XC Bartlett,231,291,38.3,7.597911


In [51]:


# Step 1: Group by match (or 'id' for match identifier) and batsman, and sum the runs
batsman_match_runs = ball_by_ball_df.groupby(['id', 'batsman'])['batsman_runs'].sum().reset_index()

# Step 2: Function to categorize runs into 30+, 50+, 100+
def categorize_runs(runs):
    if runs >= 100:
        return '100+'
    elif runs >= 50:
        return '50+'
    elif runs >= 30:
        return '30+'
    elif runs==0:
        return 'Duck'
    else:
        return 'Below 30'

# Step 3: Apply the categorization to the total runs for each match and batsman
batsman_match_runs['score_category'] = batsman_match_runs['batsman_runs'].apply(categorize_runs)

# Step 4: Count the number of occurrences of each score category per batsman across matches
score_counts = batsman_match_runs.groupby(['batsman', 'score_category']).size().unstack(fill_value=0)

# Step 5: Display the result, where the rows are batsmen and columns are score categories
print(score_counts)


score_category  100+  30+  50+  Below 30  Duck
batsman                                       
A Zampa            0    0    0         2     0
AC Agar            0    0    0         4     1
AD Hales           0    2    1         5     1
AI Ross            0    2    0         6     0
AJ Finch           0    2    0         1     1
...              ...  ...  ...       ...   ...
Usama Mir          0    0    0         1     2
W Sutherland       0    0    1         7     0
XC Bartlett        0    0    0         4     1
Z Crawley          0    0    2         4     0
Zaman Khan         0    0    0         2     0

[125 rows x 5 columns]


In [43]:
batsman_match_runs.head()

,id,batsman,batsman_runs,score_category
0,1386094,C Munro,99,50+
1,1386094,GJ Maxwell,23,Below 30
2,1386094,HWR Cartwright,33,30+
3,1386094,JA Burns,22,Below 30
4,1386094,JS Paris,5,Below 30


In [44]:

batsman_match_runs = ball_by_ball_df.groupby(['id', 'batsman'])['batsman_runs'].sum().reset_index()

batsman_runs_dict = {}

for _, row in batsman_match_runs.iterrows():
    batsman = row['batsman']
    match_id = row['id']
    runs = row['batsman_runs']
    
    # Initialize batsman entry if not already present
    if batsman not in batsman_runs_dict:
        batsman_runs_dict[batsman] = {}
    
    # Add the runs scored in this match to the batsman's dictionary
    batsman_runs_dict[batsman][match_id] = runs

# Display the dictionary
print(batsman_runs_dict)


{'C Munro': {'1386094': 99, '1386099': 46, '1386103': 12, '1386109': 0, '1386117': 9, '1386122': 56, '1386125': 2}, 'GJ Maxwell': {'1386094': 23, '1386105': 30, '1386107': 12, '1386110': 35, '1386113': 28, '1386116': 32, '1386121': 31, '1386129': 20, '1386131': 32}, 'HWR Cartwright': {'1386094': 33, '1386100': 24, '1386105': 22, '1386107': 47, '1386110': 2, '1386121': 29, '1386129': 38, '1386131': 14}, 'JA Burns': {'1386094': 22}, 'JS Paris': {'1386094': 5, '1386100': 0}, 'LA Dawson': {'1386094': 7, '1386100': 22, '1386105': 2}, 'M Bryant': {'1386094': 15, '1386128': 29, '1386134': 12, '1386136': 9, '1386137': 29}, 'M Labuschagne': {'1386094': 30, '1386125': 45}, 'MP Stoinis': {'1386094': 2, '1386100': 13, '1386105': 9, '1386107': 6, '1386110': 0, '1386113': 55, '1386121': 34, '1386129': 2, '1386131': 48}, 'NM Coulter-Nile': {'1386094': 1, '1386129': 0}, 'OP Stone': {'1386094': 2}, 'SB Harper': {'1386094': 4, '1386100': 1, '1386105': 5, '1386107': 17, '1386110': 0}, 'SW Billings': {'13

In [46]:

fours_sixes_df = ball_by_ball_df[ball_by_ball_df['batsman_runs'].isin([4, 6])]

# Step 2: Group by match ('id'), batsman, and the number of runs (either 4 or 6), and count occurrences
fours_sixes_count = fours_sixes_df.groupby(['id', 'batsman', 'batsman_runs']).size().unstack(fill_value=0)

# Step 3: Rename columns to 'fours' and 'sixes' for clarity
fours_sixes_count.columns = fours_sixes_count.columns.map({4: 'fours', 6: 'sixes'})

# Step 4: Reset index to have a clean DataFrame format
fours_sixes_count = fours_sixes_count.reset_index()

# Step 5: Fill missing values with 0 (if a batsman didn't hit a 4 or 6 in a match)
fours_sixes_count = fours_sixes_count.fillna(0)

# Step 6: Display the resulting DataFrame
print(fours_sixes_count)


batsman_runs       id         batsman  fours  sixes
0             1386094         C Munro      9      5
1             1386094      GJ Maxwell      3      1
2             1386094  HWR Cartwright      2      3
3             1386094        JA Burns      1      1
4             1386094        M Bryant      3      0
..                ...             ...    ...    ...
415           1386137      MT Renshaw      4      1
416           1386137    NA McSweeney      5      0
417           1386137       PI Walter      1      0
418           1386137       SA Abbott      1      1
419           1386137        T Murphy      1      0

[420 rows x 4 columns]


In [47]:

legit_balls_faced = ball_by_ball_df[ball_by_ball_df['extras_type'] != 'wides']

# Group by match_id and batsman to calculate total runs and balls faced
strike_rate_df = legit_balls_faced.groupby(['id', 'batsman']).agg(
    total_runs=('batsman_runs', 'sum'),      # Sum of runs scored by the batsman
    balls_faced=('ball', 'size')             # Count of legitimate balls faced
).reset_index()

# Calculate strike rate
strike_rate_df['strike_rate'] = (strike_rate_df['total_runs'] / strike_rate_df['balls_faced']) * 100

# Display the result
print(strike_rate_df[['id', 'batsman', 'total_runs', 'balls_faced', 'strike_rate']])


          id         batsman  total_runs  balls_faced  strike_rate
0    1386094         C Munro          99           61   162.295082
1    1386094      GJ Maxwell          23           14   164.285714
2    1386094  HWR Cartwright          33           16   206.250000
3    1386094        JA Burns          22           23    95.652174
4    1386094        JS Paris           5            6    83.333333
..       ...             ...         ...          ...          ...
627  1386137    NA McSweeney          33           32   103.125000
628  1386137       PI Walter           6            4   150.000000
629  1386137       SA Abbott          16           11   145.454545
630  1386137     SNJ O'Keefe           0            1     0.000000
631  1386137        T Murphy           8            9    88.888889

[632 rows x 5 columns]


In [48]:

wicket_types_for_bowler = ['caught', 'lbw', 'bowled', 'caught and bowled', 'stumped']
wickets_df = ball_by_ball_df[
    (ball_by_ball_df['is_wicket'] == 1) &
    (ball_by_ball_df['dismissal_kind'].isin(wicket_types_for_bowler))
]

wickets_per_bowler = wickets_df.groupby(['id', 'bowler']).agg(
    total_wickets=('dismissal_kind', 'size')  # Count of wickets taken by the bowler
).reset_index()

lbw_bowled_wickets = wickets_df[wickets_df['dismissal_kind'].isin(['lbw', 'bowled'])]
lbw_bowled_count = lbw_bowled_wickets.groupby(['id', 'bowler']).agg(
    lbw_bowled_wickets=('dismissal_kind', 'size')
).reset_index()

wickets_per_bowler = pd.merge(
    wickets_per_bowler, lbw_bowled_count, on=['id', 'bowler'], how='left'
).fillna(0)

wickets_per_bowler['lbw_bowled_wickets'] = wickets_per_bowler['lbw_bowled_wickets'].astype(int)

def wicket_haul_category(wickets):
    if wickets >= 5:
        return '5+ wickets'
    elif wickets == 4:
        return '4 wickets'
    elif wickets == 3:
        return '3 wickets'
    else:
        return 'Less than 3'

wickets_per_bowler['wicket_haul'] = wickets_per_bowler['total_wickets'].apply(wicket_haul_category)

print(wickets_per_bowler[['id', 'bowler', 'total_wickets', 'lbw_bowled_wickets', 'wicket_haul']])


          id        bowler  total_wickets  lbw_bowled_wickets  wicket_haul
0    1386094    GJ Maxwell              1                   0  Less than 3
1    1386094      JS Paris              1                   0  Less than 3
2    1386094      MG Neser              2                   1  Less than 3
3    1386094    MJ Swepson              3                   2    3 wickets
4    1386094  MP Kuhnemann              1                   0  Less than 3
..       ...           ...            ...                 ...          ...
286  1386137     PI Walter              1                   0  Less than 3
287  1386137     SA Abbott              4                   0    4 wickets
288  1386137    SH Johnson              4                   1    4 wickets
289  1386137   SNJ O'Keefe              1                   1  Less than 3
290  1386137   XC Bartlett              2                   0  Less than 3

[291 rows x 5 columns]


In [50]:

over_runs = ball_by_ball_df.groupby(['id', 'bowler', 'over']).agg(
    total_runs_in_over=('total_runs', 'sum')
).reset_index()

maiden_overs = over_runs[over_runs['total_runs_in_over'] == 0]

maiden_overs_count = maiden_overs.groupby(['id', 'bowler']).size().reset_index(name='maiden_overs')

print("Number of Maiden Overs per Bowler per Match:")
print(maiden_overs_count)


Number of Maiden Overs per Bowler per Match:
        id          bowler  maiden_overs
0  1386094      MJ Swepson             1
1  1386098     CJ Anderson             1
2  1386100       AM Hardie             1
3  1386100  JP Behrendorff             1
4  1386118   JA Richardson             1
5  1386119        TH David             1
6  1386123         AC Agar             1
7  1386128          AJ Tye             1
8  1386134       J Edwards             1
9  1386134      SH Johnson             1
